In [2]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.model_selection import cross_validate

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

import numpy as np
import os
import pandas as pd

# Data


In [3]:
data_path = "../data"
DATA = []
for file in os.listdir(data_path):
    DATA.append(pd.read_csv(os.path.join(data_path,file)))

In [4]:
for data in DATA:
    print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        5000 non-null   int64 
 1   job        5000 non-null   object
 2   marital    5000 non-null   object
 3   education  5000 non-null   object
 4   default    5000 non-null   object
 5   balance    5000 non-null   int64 
 6   housing    5000 non-null   object
 7   loan       5000 non-null   object
 8   contact    5000 non-null   object
 9   day        5000 non-null   int64 
 10  month      5000 non-null   object
 11  duration   5000 non-null   int64 
 12  campaign   5000 non-null   int64 
 13  pdays      5000 non-null   int64 
 14  previous   5000 non-null   int64 
 15  poutcome   5000 non-null   object
 16  y          5000 non-null   object
dtypes: int64(7), object(10)
memory usage: 664.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 

# Pipeline

In [5]:
num_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer()),
    ('scale', MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy="most_frequent")),
    ('one-hot', OneHotEncoder(handle_unknown='ignore'))
])


col_trans = ColumnTransformer([
    ('num_pipeline', num_pipeline, make_column_selector(dtype_include = np.number)),
    ('cat_pipeline', cat_pipeline, make_column_selector(dtype_include = np.object_))
])


In [7]:
classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    XGBClassifier(),
]

pipelines = []

In [8]:
for classifier in classifiers:
    pipelines.append((type(classifier), Pipeline([("transformer", col_trans), ("model", classifier)])))

In [9]:
from sklearn.calibration import LabelEncoder


scores = []

for pipe in pipelines:
    pipe_score = []
    for data in DATA:
        score = cross_validate(pipe[1], data.iloc[:, :-1], LabelEncoder().fit_transform(data.iloc[:,-1]), cv = 5, scoring="accuracy")
        pipe_score.append(score["test_score"].mean())

    scores.append((pipe[0], pipe_score))

In [10]:
os.listdir("../data")

['banking_final.csv',
 'flights_final.csv',
 'mushrooms_final.csv',
 'weather_final.csv']

In [11]:
scores

[(sklearn.tree._classes.DecisionTreeClassifier,
  [0.8667999999999999, 0.9136, 0.9798, 0.7933999999999999]),
 (sklearn.ensemble._forest.RandomForestClassifier,
  [0.901, 0.9426, 0.9998000000000001, 0.8324]),
 (xgboost.sklearn.XGBClassifier,
  [0.9016, 0.9469999999999998, 0.9974000000000001, 0.8368])]

# Random Searching

In [18]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

best_model = []
best_score = []

param_distributions = [
    {
        "model__max_depth": randint(1,30),
        "model__min_samples_split":randint(1,60),
        "model__criterion":["gini","entropy"],
        "model__min_samples_leaf":randint(1,60)
    }
]
best_params_decisionTree = [[],[],[],[]]
for i,pipe in enumerate(pipelines):
    if i >0: break
    pipe_best_models = []
    pipe_best_scores = []
    for j,data in enumerate(DATA):
        #data.iloc[:, :-1],LabelEncoder().fit_transform(data.iloc[:,-1]),
        rs = RandomizedSearchCV(pipe[1], 
                                param_distributions= param_distributions[i]
                                )
        rs.fit(data.iloc[:, :-1],LabelEncoder().fit_transform(data.iloc[:,-1]))
        best_params_decisionTree[j].append(rs.best_params_)
        

In [19]:
best_params_decisionTree

[[{'model__criterion': 'entropy',
   'model__max_depth': 5,
   'model__min_samples_leaf': 59,
   'model__min_samples_split': 27}],
 [{'model__criterion': 'gini',
   'model__max_depth': 14,
   'model__min_samples_leaf': 11,
   'model__min_samples_split': 43}],
 [{'model__criterion': 'entropy',
   'model__max_depth': 27,
   'model__min_samples_leaf': 1,
   'model__min_samples_split': 15}],
 [{'model__criterion': 'gini',
   'model__max_depth': 7,
   'model__min_samples_leaf': 43,
   'model__min_samples_split': 44}]]